# Rules for League

## Batting
* Batting AVG (ratio of a batter's safe hits per official times at bat)
* Runs Scored R
* Stolen Bases SB
* Home Runs HR
* Runs Batted In RBI

## Pitching
* Earned Run Average ERA
* Walks plus Hits Per Innings Pitched WHIP
* Saves SV
* Strikeouts K
* Wins W

## TODO
* Find definition of WHIP (walks, hits per inning pitched)
* Add WHIP to Pitchers
* Migrate load from local file to online CSV from Github
* Find stddev / top quartile lists for each rule

Ideas
* correlate pitch type to pitcher statistics

2.2 Batting Table

playerID       Player ID code
yearID         Year
stint          player's stint (order of appearances within a season)
teamID         Team
lgID           League
G              Games
AB             At Bats
R              Runs
H              Hits
2B             Doubles
3B             Triples
HR             Homeruns
RBI            Runs Batted In
SB             Stolen Bases
CS             Caught Stealing
BB             Base on Balls
SO             Strikeouts
IBB            Intentional walks
HBP            Hit by pitch
SH             Sacrifice hits
SF             Sacrifice flies
GIDP           Grounded into double plays

2.3 Pitching table

playerID       Player ID code
yearID         Year
stint          player's stint (order of appearances within a season)
teamID         Team
lgID           League
W              Wins
L              Losses
G              Games
GS             Games Started
CG             Complete Games 
SHO            Shutouts
SV             Saves
IPOuts         Outs Pitched (innings pitched x 3)
H              Hits
ER             Earned Runs
HR             Homeruns
BB             Walks
SO             Strikeouts
BAOpp          Opponent's Batting Average
ERA            Earned Run Average
IBB            Intentional Walks
WP             Wild Pitches
HBP            Batters Hit By Pitch
BK             Balks
BFP            Batters faced by Pitcher
GF             Games Finished
R              Runs Allowed
SH             Sacrifices by opposing batters
SF             Sacrifice flies by opposing batters
GIDP           Grounded into double plays by opposing batter

2.4 Fielding Table

playerID       Player ID code
yearID         Year
stint          player's stint (order of appearances within a season)
teamID         Team
lgID           League
Pos            Position
G              Games 
GS             Games Started
InnOuts        Time played in the field expressed as outs 
PO             Putouts
A              Assists
E              Errors
DP             Double Plays
PB             Passed Balls (by catchers)
WP             Wild Pitches (by catchers)
SB             Opponent Stolen Bases (by catchers)
CS             Opponents Caught Stealing (by catchers)
ZR             Zone Rating

In [1]:
import mlbgame
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import os
year = 2010

bball_directory = "/Users/cmcavoy/projects/lahman/baseballdatabank-2017.1/core/"

teams = pd.read_csv(os.path.join(bball_directory,'Teams.csv'))
teams = teams[teams['yearID'] >= 1985]
teams = teams[['yearID', 'teamID', 'Rank', 'R', 'RA', 'G', 'W', 'H', 'BB', 'HBP', 'AB', 'SF', 'HR', '2B', '3B']]
teams = teams.set_index(['yearID', 'teamID'])

salaries = pd.read_csv(os.path.join(bball_directory, 'Salaries.csv'))
salaries_by_yearID_teamID = salaries.groupby(['yearID', 'teamID'])['salary'].sum()
teams = teams.join(salaries_by_yearID_teamID)

pitchers = pd.read_csv(os.path.join(bball_directory, 'Pitching.csv'))
master = pd.read_csv(os.path.join(bball_directory, 'Master.csv'))
m = master[['playerID', 'nameFirst', 'nameLast']]
teams = pd.read_csv(os.path.join(bball_directory, 'Teams.csv'))
t = teams[teams["yearID"] == 2016][["teamID", "name"]]
batting = pd.read_csv(os.path.join(bball_directory, 'Batting.csv'))

pitchers = pd.merge(pitchers, m, how="outer", on='playerID')
pitchers = pitchers[pitchers['yearID'] >= year]
pitchers.set_index(['yearID', 'teamID', 'nameLast'])

batters = pd.merge(batting, m, how="outer", on='playerID')
batters = batters[batters['yearID'] >= year]
batters['SLUG'] = (batters["H"] + batters["2B"] + (batters["3B"]*2) + (batters["HR"]*3)) / batters["AB"]
batters = pd.merge(batters, t, how="outer", on="teamID")
batters = batters.rename(columns={"name":"teamName"})
batters["BA"] = batters["H"] / batters["AB"]
batters.set_index(['yearID', 'teamID', 'nameLast'])

#p = p[p['yearID'] >= 2010] # all pitcher stats


playerID  stint lgID      G     AB      R      H  \
yearID teamID nameLast                                                         
2010.0 PHI    Moyer        moyerja01    1.0   NL   19.0   26.0    1.0    2.0   
2012.0 PHI    Thome        thomeji01    1.0   NL   30.0   62.0    9.0   15.0   
2010.0 PHI    Castro       castrju01    1.0   NL   54.0  126.0    7.0   25.0   
              Sweeney      sweenmi01    2.0   NL   26.0   52.0   10.0   12.0   
              Ibanez       ibanera01    1.0   NL  155.0  561.0   75.0  154.0   
2011.0 PHI    Ibanez       ibanera01    1.0   NL  144.0  535.0   65.0  131.0   
2010.0 PHI    Halladay     hallaro01    1.0   NL   33.0   92.0    6.0   13.0   
2011.0 PHI    Halladay     hallaro01    1.0   NL   32.0   74.0    3.0    9.0   
2012.0 PHI    Halladay     hallaro01    1.0   NL   25.0   53.0    4.0    9.0   
2013.0 PHI    Halladay     hallaro01    1.0   NL   13.0   18.0    0.0    0.0   
2010.0 PHI    Polanco      polanpl01    1.0   NL  132.0  554.0   76.0  165.0   
2011.0 PHI    Polanco      polanpl01    1.0   NL  122.0  469.0   46.0  130.0   
2012.0 PHI    Polanco      polanpl01    1.0   NL   90.0  303.0   28.0   78.0   
2014.0 PHI    Burnett      burneaj01    1.0   NL   34.0   60.0    5.0    8.0   
2010.0 PHI    Durbin       durbich01    1.0   NL   64.0    2.0    0.0    0.0   
2013.0 PHI    Durbin       durbich01    1.0   NL   16.0    0.0    0.0    0.0   
              McDonald     mcdonjo03    3.0   NL   21.0   23.0    5.0    4.0   
2010.0 PHI    Romero       romerjc01    1.0   NL   60.0    0.0    0.0    0.0   
2011.0 PHI    Romero       romerjc01    1.0   NL   24.0    0.0    0.0    0.0   
2010.0 PHI    Figueroa     figuene01    1.0   NL   13.0    4.0    1.0    2.0   
              Gload        gloadro01    1.0   NL   94.0  128.0   16.0   36.0   
2011.0 PHI    Gload        gloadro01    1.0   NL   93.0  113.0    3.0   29.0   
2012.0 PHI    Pierre       pierrju01    1.0   NL  130.0  394.0   59.0  121.0   
2010.0 PHI    Rollins      rolliji01    1.0   NL   88.0  350.0   48.0   85.0   
2011.0 PHI    Rollins      rolliji01    1.0   NL  142.0  567.0   87.0  152.0   
2012.0 PHI    Rollins      rolliji01    1.0   NL  156.0  632.0  102.0  158.0   
2013.0 PHI    Rollins      rolliji01    1.0   NL  160.0  600.0   65.0  151.0   
2014.0 PHI    Rollins      rolliji01    1.0   NL  138.0  538.0   78.0  131.0   
2010.0 PHI    Schneider    schnebr01    1.0   NL   47.0  125.0   17.0   30.0   
2011.0 PHI    Schneider    schnebr01    1.0   NL   41.0  125.0   11.0   22.0   
...                              ...    ...  ...    ...    ...    ...    ...   
2015.0 MIA    Rojas        rojasmi02    1.0   NL   60.0  142.0   13.0   40.0   
2016.0 MIA    Rojas        rojasmi02    1.0   NL  123.0  194.0   27.0   48.0   
              Scruggs      scrugxa01    1.0   NL   24.0   62.0    1.0   13.0   
2015.0 MIA    Telis        telisto01    2.0   NL   17.0   27.0    1.0    4.0   
2016.0 MIA    Telis        telisto01    1.0   NL   10.0   13.0    1.0    4.0   
2015.0 MIA    Barraclough  barraky01    1.0   NL   25.0    0.0    0.0    0.0   
2016.0 MIA    Barraclough  barraky01    1.0   NL   75.0    1.0    0.0    0.0   
2015.0 MIA    Conley       conlead01    1.0   NL   15.0   18.0    0.0    4.0   
2016.0 MIA    Conley       conlead01    1.0   NL   25.0   41.0    4.0    5.0   
2015.0 MIA    Ellington    ellinbr02    1.0   NL   23.0    0.0    0.0    0.0   
2016.0 MIA    Ellington    ellinbr02    1.0   NL   32.0    1.0    0.0    0.0   
2015.0 MIA    Flores       floreke01    1.0   NL    7.0    1.0    0.0    0.0   
2016.0 MIA    Flores       floreke01    1.0   NL    1.0    1.0    0.0    0.0   
              Hall          hallco01    1.0   NL    2.0    0.0    0.0    0.0   
2015.0 MIA    Lazo          lazora01    1.0   NL    7.0    0.0    0.0    0.0   
              McGough      mcgousc01    1.0   NL    6.0    0.0    0.0    0.0   
              Nicolino     nicolju01    1.0   NL   13.0   25.0    0.0    1.0   
2016.0 MIA    Nicolino     n

In [2]:
batters.loc[batters["yearID"] == 2016][batters["teamID"] == "CHN"][batters["H"]>0][["AB", "H", "2B", "3B", "HR"]]

/Users/cmcavoy/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


,AB,H,2B,3B,HR
9408,63.0,6.0,2.0,0.0,0.0
9411,166.0,38.0,6.0,0.0,10.0
9459,65.0,16.0,3.0,0.0,0.0
9461,59.0,6.0,3.0,0.0,0.0
9469,241.0,52.0,8.0,1.0,8.0
9475,523.0,142.0,31.0,3.0,18.0
9492,456.0,126.0,25.0,7.0,13.0
9528,8.0,1.0,0.0,0.0,0.0
9532,103.0,26.0,7.0,2.0,1.0
9548,65.0,17.0,2.0,1.0,2.0


In [3]:
batters.set_index(['yearID', 'teamID', 'nameLast'])

playerID  stint lgID      G     AB      R      H  \
yearID teamID nameLast                                                         
2010.0 PHI    Moyer        moyerja01    1.0   NL   19.0   26.0    1.0    2.0   
2012.0 PHI    Thome        thomeji01    1.0   NL   30.0   62.0    9.0   15.0   
2010.0 PHI    Castro       castrju01    1.0   NL   54.0  126.0    7.0   25.0   
              Sweeney      sweenmi01    2.0   NL   26.0   52.0   10.0   12.0   
              Ibanez       ibanera01    1.0   NL  155.0  561.0   75.0  154.0   
2011.0 PHI    Ibanez       ibanera01    1.0   NL  144.0  535.0   65.0  131.0   
2010.0 PHI    Halladay     hallaro01    1.0   NL   33.0   92.0    6.0   13.0   
2011.0 PHI    Halladay     hallaro01    1.0   NL   32.0   74.0    3.0    9.0   
2012.0 PHI    Halladay     hallaro01    1.0   NL   25.0   53.0    4.0    9.0   
2013.0 PHI    Halladay     hallaro01    1.0   NL   13.0   18.0    0.0    0.0   
2010.0 PHI    Polanco      polanpl01    1.0   NL  132.0  554.0   76.0  165.0   
2011.0 PHI    Polanco      polanpl01    1.0   NL  122.0  469.0   46.0  130.0   
2012.0 PHI    Polanco      polanpl01    1.0   NL   90.0  303.0   28.0   78.0   
2014.0 PHI    Burnett      burneaj01    1.0   NL   34.0   60.0    5.0    8.0   
2010.0 PHI    Durbin       durbich01    1.0   NL   64.0    2.0    0.0    0.0   
2013.0 PHI    Durbin       durbich01    1.0   NL   16.0    0.0    0.0    0.0   
              McDonald     mcdonjo03    3.0   NL   21.0   23.0    5.0    4.0   
2010.0 PHI    Romero       romerjc01    1.0   NL   60.0    0.0    0.0    0.0   
2011.0 PHI    Romero       romerjc01    1.0   NL   24.0    0.0    0.0    0.0   
2010.0 PHI    Figueroa     figuene01    1.0   NL   13.0    4.0    1.0    2.0   
              Gload        gloadro01    1.0   NL   94.0  128.0   16.0   36.0   
2011.0 PHI    Gload        gloadro01    1.0   NL   93.0  113.0    3.0   29.0   
2012.0 PHI    Pierre       pierrju01    1.0   NL  130.0  394.0   59.0  121.0   
2010.0 PHI    Rollins      rolliji01    1.0   NL   88.0  350.0   48.0   85.0   
2011.0 PHI    Rollins      rolliji01    1.0   NL  142.0  567.0   87.0  152.0   
2012.0 PHI    Rollins      rolliji01    1.0   NL  156.0  632.0  102.0  158.0   
2013.0 PHI    Rollins      rolliji01    1.0   NL  160.0  600.0   65.0  151.0   
2014.0 PHI    Rollins      rolliji01    1.0   NL  138.0  538.0   78.0  131.0   
2010.0 PHI    Schneider    schnebr01    1.0   NL   47.0  125.0   17.0   30.0   
2011.0 PHI    Schneider    schnebr01    1.0   NL   41.0  125.0   11.0   22.0   
...                              ...    ...  ...    ...    ...    ...    ...   
2015.0 MIA    Rojas        rojasmi02    1.0   NL   60.0  142.0   13.0   40.0   
2016.0 MIA    Rojas        rojasmi02    1.0   NL  123.0  194.0   27.0   48.0   
              Scruggs      scrugxa01    1.0   NL   24.0   62.0    1.0   13.0   
2015.0 MIA    Telis        telisto01    2.0   NL   17.0   27.0    1.0    4.0   
2016.0 MIA    Telis        telisto01    1.0   NL   10.0   13.0    1.0    4.0   
2015.0 MIA    Barraclough  barraky01    1.0   NL   25.0    0.0    0.0    0.0   
2016.0 MIA    Barraclough  barraky01    1.0   NL   75.0    1.0    0.0    0.0   
2015.0 MIA    Conley       conlead01    1.0   NL   15.0   18.0    0.0    4.0   
2016.0 MIA    Conley       conlead01    1.0   NL   25.0   41.0    4.0    5.0   
2015.0 MIA    Ellington    ellinbr02    1.0   NL   23.0    0.0    0.0    0.0   
2016.0 MIA    Ellington    ellinbr02    1.0   NL   32.0    1.0    0.0    0.0   
2015.0 MIA    Flores       floreke01    1.0   NL    7.0    1.0    0.0    0.0   
2016.0 MIA    Flores       floreke01    1.0   NL    1.0    1.0    0.0    0.0   
              Hall          hallco01    1.0   NL    2.0    0.0    0.0    0.0   
2015.0 MIA    Lazo          lazora01    1.0   NL    7.0    0.0    0.0    0.0   
              McGough      mcgousc01    1.0   NL    6.0    0.0    0.0    0.0   
              Nicolino     nicolju01    1.0   NL   13.0   25.0    0.0    1.0   
2016.0 MIA    Nicolino     n

In [4]:
batters.columns

Index(['playerID', 'yearID', 'stint', 'teamID', 'lgID', 'G', 'AB', 'R', 'H',
       '2B', '3B', 'HR', 'RBI', 'SB', 'CS', 'BB', 'SO', 'IBB', 'HBP', 'SH',
       'SF', 'GIDP', 'nameFirst', 'nameLast', 'SLUG', 'teamName', 'BA'],
      dtype='object')

In [5]:
batters["SLUG"] = batters["2B"] + batters["3B"] + batters["HR"]

In [6]:
batters["SLUG"]

0        0.0
1        7.0
2        5.0
3        4.0
4       58.0
5       52.0
6        0.0
7        2.0
8        0.0
9        0.0
10      35.0
11      19.0
12      17.0
13       2.0
14       0.0
15       0.0
16       1.0
17       0.0
18       0.0
19       1.0
20      14.0
21       8.0
22      17.0
23      27.0
24      40.0
25      61.0
26      44.0
27      43.0
28       9.0
29       6.0
        ... 
9936     9.0
9937    13.0
9938     4.0
9939     0.0
9940     1.0
9941     0.0
9942     0.0
9943     0.0
9944     0.0
9945     0.0
9946     0.0
9947     0.0
9948     0.0
9949     0.0
9950     0.0
9951     0.0
9952     0.0
9953     0.0
9954     0.0
9955     0.0
9956     0.0
9957     0.0
9958     0.0
9959     0.0
9960     0.0
9961     0.0
9962     0.0
9963     2.0
9964     1.0
9965     0.0
Name: SLUG, dtype: float64

In [7]:
batters.loc[batters["yearID"] == 2016][batters["teamID"]=="CHN"][batters["H"]>0][["nameLast", "teamID", "AB", "H", "2B", "3B", "HR"]].sort_values("AB")

/Users/cmcavoy/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


,nameLast,teamID,AB,H,2B,3B,HR
9582,Kalish,CHN,7.0,2.0,0.0,0.0,0.0
9528,Cahill,CHN,8.0,1.0,0.0,0.0,0.0
9708,Montgomery,CHN,11.0,1.0,0.0,0.0,0.0
9714,Candelario,CHN,11.0,1.0,0.0,0.0,0.0
9605,Wood,CHN,11.0,2.0,0.0,0.0,0.0
9642,Kawasaki,CHN,21.0,7.0,2.0,0.0,0.0
9615,Federowicz,CHN,31.0,6.0,2.0,0.0,0.0
9686,Hendricks,CHN,58.0,8.0,0.0,0.0,0.0
9461,Lester,CHN,59.0,6.0,3.0,0.0,0.0
9408,Lackey,CHN,63.0,6.0,2.0,0.0,0.0


In [8]:
batters['SLUG'] = (batters["H"] + batters["2B"] + (batters["3B"]*2) + (batters["HR"]*3)) / batters["AB"]

In [9]:
batters[batters["nameLast"] == 'Bryant'][["nameLast", "yearID", "SLUG"]]

,nameLast,yearID,SLUG
9704,Bryant,2015.0,0.488372
9705,Bryant,2016.0,0.553897


In [10]:
batters[batters["yearID"] == 2016][batters["AB"] >= 600][["nameLast", "SLUG", "AB", "teamID"]].sort_values("SLUG")

/Users/cmcavoy/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


,nameLast,SLUG,AB,teamID
8216,Escobar,0.350078,637.0,KCA
9777,Prado,0.416667,600.0,MIA
1215,Eaton,0.428110,619.0,CHA
8319,Hosmer,0.433058,605.0,KCA
3579,Lindor,0.435430,604.0,CLE
3675,Jones,0.436187,619.0,BAL
5233,Bogaerts,0.446319,652.0,BOS
2138,Desmond,0.446400,625.0,TEX
5062,Pedroia,0.448657,633.0,BOS
3898,Schoop,0.453659,615.0,BAL


In [11]:
teams = pd.read_csv(os.path.join(bball_directory, 'Teams.csv'))

In [12]:
teams.columns

Index(['yearID', 'lgID', 'teamID', 'franchID', 'divID', 'Rank', 'G', 'Ghome',
       'W', 'L', 'DivWin', 'WCWin', 'LgWin', 'WSWin', 'R', 'AB', 'H', '2B',
       '3B', 'HR', 'BB', 'SO', 'SB', 'CS', 'HBP', 'SF', 'RA', 'ER', 'ERA',
       'CG', 'SHO', 'SV', 'IPouts', 'HA', 'HRA', 'BBA', 'SOA', 'E', 'DP', 'FP',
       'name', 'park', 'attendance', 'BPF', 'PPF', 'teamIDBR',
       'teamIDlahman45', 'teamIDretro'],
      dtype='object')

In [13]:
t = teams[["teamID", "name"]][teams["yearID"]>=2010]

In [14]:
batters = batters.merge(t, on="teamID")

In [15]:
batters.columns

Index(['playerID', 'yearID', 'stint', 'teamID', 'lgID', 'G', 'AB', 'R', 'H',
       '2B', '3B', 'HR', 'RBI', 'SB', 'CS', 'BB', 'SO', 'IBB', 'HBP', 'SH',
       'SF', 'GIDP', 'nameFirst', 'nameLast', 'SLUG', 'teamName', 'BA',
       'name'],
      dtype='object')

In [16]:
batters = batters.rename(columns={"name":"teamName"})

In [17]:
batters.columns

Index(['playerID', 'yearID', 'stint', 'teamID', 'lgID', 'G', 'AB', 'R', 'H',
       '2B', '3B', 'HR', 'RBI', 'SB', 'CS', 'BB', 'SO', 'IBB', 'HBP', 'SH',
       'SF', 'GIDP', 'nameFirst', 'nameLast', 'SLUG', 'teamName', 'BA',
       'teamName'],
      dtype='object')

In [18]:
batting[batting["playerID"] == 'fowlede01']

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
90486,fowlede01,2008,1,COL,NL,13,26,3,4,0,...,0.0,0.0,1.0,0,5.0,0.0,1.0,0.0,0.0,0.0
91861,fowlede01,2009,1,COL,NL,135,433,73,115,29,...,34.0,27.0,10.0,67,116.0,1.0,1.0,14.0,3.0,4.0
93241,fowlede01,2010,1,COL,NL,132,439,73,114,20,...,36.0,13.0,7.0,57,104.0,0.0,2.0,7.0,0.0,5.0
94608,fowlede01,2011,1,COL,NL,125,481,84,128,35,...,45.0,12.0,9.0,68,130.0,3.0,6.0,7.0,1.0,6.0
96010,fowlede01,2012,1,COL,NL,143,454,72,136,18,...,53.0,12.0,5.0,68,128.0,1.0,0.0,6.0,2.0,5.0
97425,fowlede01,2013,1,COL,NL,119,415,71,109,18,...,42.0,19.0,9.0,65,105.0,1.0,6.0,4.0,2.0,5.0
98834,fowlede01,2014,1,HOU,AL,116,434,61,120,21,...,35.0,11.0,4.0,66,108.0,2.0,3.0,1.0,1.0,6.0
100287,fowlede01,2015,1,CHN,NL,156,596,102,149,29,...,46.0,20.0,7.0,84,154.0,1.0,5.0,2.0,3.0,4.0
101762,fowlede01,2016,1,CHN,NL,125,456,84,126,25,...,48.0,13.0,4.0,79,124.0,0.0,11.0,1.0,4.0,3.0


In [19]:
batters.to_csv("batters_with_slug_and_teams.csv")

In [20]:
batters_m = pd.merge(batting, m, how="outer", on='playerID')

In [21]:
batters[batters["teamID"] == 'CHN'][batters["AB"] >500][['yearID','nameLast', 'SLUG']].plot(x="yearID", y="SLUG")

/Users/cmcavoy/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


In [22]:
batters_t = pd.merge(t, batters_m, on="teamID")

In [23]:
batters_t[batters_t["playerID"] == 'fowlede01']

,teamID,name,playerID,yearID,stint,lgID,G,AB,R,H,...,CS,BB,SO,IBB,HBP,SH,SF,GIDP,nameFirst,nameLast
103752,CHN,Chicago Cubs,fowlede01,2015.0,1.0,NL,156.0,596.0,102.0,149.0,...,7.0,84.0,154.0,1.0,5.0,2.0,3.0,4.0,Dexter,Fowler
103753,CHN,Chicago Cubs,fowlede01,2016.0,1.0,NL,125.0,456.0,84.0,126.0,...,4.0,79.0,124.0,0.0,11.0,1.0,4.0,3.0,Dexter,Fowler
108615,CHN,Chicago Cubs,fowlede01,2015.0,1.0,NL,156.0,596.0,102.0,149.0,...,7.0,84.0,154.0,1.0,5.0,2.0,3.0,4.0,Dexter,Fowler
108616,CHN,Chicago Cubs,fowlede01,2016.0,1.0,NL,125.0,456.0,84.0,126.0,...,4.0,79.0,124.0,0.0,11.0,1.0,4.0,3.0,Dexter,Fowler
113478,CHN,Chicago Cubs,fowlede01,2015.0,1.0,NL,156.0,596.0,102.0,149.0,...,7.0,84.0,154.0,1.0,5.0,2.0,3.0,4.0,Dexter,Fowler
113479,CHN,Chicago Cubs,fowlede01,2016.0,1.0,NL,125.0,456.0,84.0,126.0,...,4.0,79.0,124.0,0.0,11.0,1.0,4.0,3.0,Dexter,Fowler
118341,CHN,Chicago Cubs,fowlede01,2015.0,1.0,NL,156.0,596.0,102.0,149.0,...,7.0,84.0,154.0,1.0,5.0,2.0,3.0,4.0,Dexter,Fowler
118342,CHN,Chicago Cubs,fowlede01,2016.0,1.0,NL,125.0,456.0,84.0,126.0,...,4.0,79.0,124.0,0.0,11.0,1.0,4.0,3.0,Dexter,Fowler
123204,CHN,Chicago Cubs,fowlede01,2015.0,1.0,NL,156.0,596.0,102.0,149.0,...,7.0,84.0,154.0,1.0,5.0,2.0,3.0,4.0,Dexter,Fowler
123205,CHN,Chicago Cubs,fowlede01,2016.0,1.0,NL,125.0,456.0,84.0,126.0,...,4.0,79.0,124.0,0.0,11.0,1.0,4.0,3.0,Dexter,Fowler


In [24]:
t

,teamID,name
2625,ARI,Arizona Diamondbacks
2626,ATL,Atlanta Braves
2627,BAL,Baltimore Orioles
2628,BOS,Boston Red Sox
2629,CHA,Chicago White Sox
2630,CHN,Chicago Cubs
2631,CIN,Cincinnati Reds
2632,CLE,Cleveland Indians
2633,COL,Colorado Rockies
2634,DET,Detroit Tigers


In [25]:
t[t["teamID"] == "SDN"]

,teamID,name
2647,SDN,San Diego Padres
2677,SDN,San Diego Padres
2707,SDN,San Diego Padres
2737,SDN,San Diego Padres
2767,SDN,San Diego Padres
2797,SDN,San Diego Padres
2827,SDN,San Diego Padres


In [26]:
t = teams[teams["yearID"]>=year][["teamID", "name", "yearID"]]

In [27]:
t[t["teamID"] == "SDN"]

,teamID,name,yearID
2647,SDN,San Diego Padres,2010
2677,SDN,San Diego Padres,2011
2707,SDN,San Diego Padres,2012
2737,SDN,San Diego Padres,2013
2767,SDN,San Diego Padres,2014
2797,SDN,San Diego Padres,2015
2827,SDN,San Diego Padres,2016


In [28]:
t = teams[teams["yearID"] == 2016][["teamID", "name"]]

In [29]:
batters[batters["teamID"] == 'CHN'][batters["AB"] >500][['yearID','nameLast', 'SLUG', 'BA']]

/Users/cmcavoy/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


,yearID,nameLast,SLUG,BA
65204,2011.0,Ramirez,0.509735,0.306195
65205,2011.0,Ramirez,0.509735,0.306195
65206,2011.0,Ramirez,0.509735,0.306195
65207,2011.0,Ramirez,0.509735,0.306195
65208,2011.0,Ramirez,0.509735,0.306195
65209,2011.0,Ramirez,0.509735,0.306195
65210,2011.0,Ramirez,0.509735,0.306195
65267,2012.0,Soriano,0.499109,0.262032
65268,2012.0,Soriano,0.499109,0.262032
65269,2012.0,Soriano,0.499109,0.262032


In [30]:
batters.columns

Index(['playerID', 'yearID', 'stint', 'teamID', 'lgID', 'G', 'AB', 'R', 'H',
       '2B', '3B', 'HR', 'RBI', 'SB', 'CS', 'BB', 'SO', 'IBB', 'HBP', 'SH',
       'SF', 'GIDP', 'nameFirst', 'nameLast', 'SLUG', 'teamName', 'BA',
       'teamName'],
      dtype='object')

In [31]:
batters["BA"] = batters["H"] / batters["AB"]

In [32]:
pitchers

,playerID,yearID,stint,teamID,lgID,W,L,G,GS,CG,...,HBP,BK,BFP,GF,R,SH,SF,GIDP,nameFirst,nameLast
27962,moyerja01,2010.0,1.0,PHI,NL,9.0,9.0,19.0,19.0,2.0,...,6.0,0.0,460.0,0.0,64.0,3.0,1.0,5.0,Jamie,Moyer
27963,moyerja01,2012.0,1.0,COL,NL,2.0,5.0,10.0,10.0,0.0,...,2.0,0.0,254.0,0.0,40.0,4.0,1.0,9.0,Jamie,Moyer
30798,rhodear01,2010.0,1.0,CIN,NL,4.0,4.0,69.0,0.0,0.0,...,1.0,0.0,217.0,8.0,14.0,2.0,2.0,2.0,Arthur,Rhodes
30799,rhodear01,2011.0,1.0,TEX,AL,3.0,3.0,32.0,0.0,0.0,...,0.0,0.0,105.0,10.0,13.0,0.0,0.0,4.0,Arthur,Rhodes
30800,rhodear01,2011.0,2.0,SLN,NL,0.0,1.0,19.0,0.0,0.0,...,0.0,0.0,33.0,1.0,4.0,0.0,1.0,1.0,Arthur,Rhodes
31014,batismi01,2010.0,1.0,WAS,NL,1.0,2.0,58.0,1.0,0.0,...,5.0,0.0,350.0,18.0,36.0,1.0,2.0,11.0,Miguel,Batista
31015,batismi01,2011.0,1.0,SLN,NL,3.0,2.0,26.0,1.0,0.0,...,4.0,0.0,137.0,2.0,20.0,2.0,0.0,3.0,Miguel,Batista
31016,batismi01,2011.0,2.0,NYN,NL,2.0,0.0,9.0,4.0,1.0,...,2.0,0.0,125.0,0.0,9.0,2.0,1.0,6.0,Miguel,Batista
31017,batismi01,2012.0,1.0,NYN,NL,1.0,3.0,30.0,5.0,0.0,...,1.0,0.0,220.0,7.0,28.0,3.0,2.0,3.0,Miguel,Batista
31018,batismi01,2012.0,2.0,ATL,NL,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,24.0,4.0,2.0,0.0,0.0,0.0,Miguel,Batista


In [33]:
batters.quantile

<bound method DataFrame.quantile of         playerID  yearID  stint teamID lgID      G     AB     R      H    2B  \
0      moyerja01  2010.0    1.0    PHI   NL   19.0   26.0   1.0    2.0   0.0   
1      moyerja01  2010.0    1.0    PHI   NL   19.0   26.0   1.0    2.0   0.0   
2      moyerja01  2010.0    1.0    PHI   NL   19.0   26.0   1.0    2.0   0.0   
3      moyerja01  2010.0    1.0    PHI   NL   19.0   26.0   1.0    2.0   0.0   
4      moyerja01  2010.0    1.0    PHI   NL   19.0   26.0   1.0    2.0   0.0   
5      moyerja01  2010.0    1.0    PHI   NL   19.0   26.0   1.0    2.0   0.0   
6      moyerja01  2010.0    1.0    PHI   NL   19.0   26.0   1.0    2.0   0.0   
7      thomeji01  2012.0    1.0    PHI   NL   30.0   62.0   9.0   15.0   2.0   
8      thomeji01  2012.0    1.0    PHI   NL   30.0   62.0   9.0   15.0   2.0   
9      thomeji01  2012.0    1.0    PHI   NL   30.0   62.0   9.0   15.0   2.0   
10     thomeji01  2012.0    1.0    PHI   NL   30.0   62.0   9.0   15.0   2.0   
11  

In [34]:
batters['SB'].describe()

count    68763.000000
mean         2.005308
std          5.613269
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max         68.000000
Name: SB, dtype: float64

In [35]:
batters['RBI'].describe()

count    68763.000000
mean        13.922386
std         24.322392
min          0.000000
25%          0.000000
50%          1.000000
75%         17.000000
max        139.000000
Name: RBI, dtype: float64

In [36]:
batters[batters['RBI'] > 86][['yearID', 'RBI', 'nameLast']]

,yearID,RBI,nameLast
581,2010.0,108.0,Howard
582,2010.0,108.0,Howard
583,2010.0,108.0,Howard
584,2010.0,108.0,Howard
585,2010.0,108.0,Howard
586,2010.0,108.0,Howard
587,2010.0,108.0,Howard
588,2011.0,116.0,Howard
589,2011.0,116.0,Howard
590,2011.0,116.0,Howard


In [37]:
pitchers[['yearID', 'ERA', 'nameLast']]

,yearID,ERA,nameLast
27962,2010.0,4.84,Moyer
27963,2012.0,5.70,Moyer
30798,2010.0,2.29,Rhodes
30799,2011.0,4.81,Rhodes
30800,2011.0,4.15,Rhodes
31014,2010.0,3.70,Batista
31015,2011.0,4.60,Batista
31016,2011.0,2.64,Batista
31017,2012.0,4.82,Batista
31018,2012.0,3.00,Batista
